# Hacker News Posts Analysis
Hacker News is a site started by the startup incubator Y Combinator, where user-submitted stories (known as "posts") are voted and commented upon.
We have data set with posts that has been redused from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions.

Below are descriptions of the columns:
* *id*: The unique identifier from Hacker News for the post
* *title*: The title of the post
* *url*: The URL that the posts links to, if the post has a URL
* *num_points*: The number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes
* *num_comments*: The number of comments that were made on the post
* *author*: The username of the person who submitted the post
* *created_at*: The date and time at which the post was submitted 



We're specifically interested in posts whose titles begin with either Ask HN or Show HN. Users submit Ask HN posts to ask the Hacker News community a specific question. For example, "Ask HN: How to improve my personal website?"

We'll compare these two types of posts to determine the following:

* Do Ask HN or Show HN receive more comments on average?
* Do posts created at a certain time receive more comments on average?

Let's read the file and look at first five rows:

In [5]:
from csv import reader 
opened_file = open('hacker_news.csv') 
read_file = reader(opened_file) 
hn = list(read_file)

for row in hn[0:4]:
    print(row)

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']
['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']
['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']
['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']


In order to analyze our data, we need to first remove the row containing the column headers.

In [6]:
headers = hn[0]
hn.remove(headers)
print(headers)

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


In [7]:
for row in hn[0:4]:
    print(row)

['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']
['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']
['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']
['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']


So now our data set doesn't have headers.

Since we're only concerned with post titles beginning with "Ask HN" or "Show HN", we'll create new lists of lists containing just the data for those titles.

In [8]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    title = title.lower()
    if title.startswith('ask hn'): 
        ask_posts.append(row)
    elif title.startswith('show hn'): 
        show_posts.append(row)
    else: 
        other_posts.append(row)

print("Number of 'Ask HN' posts:", len(ask_posts))
print("Number of 'Show HN' posts:", len(show_posts))
print("Number of other posts:", len(other_posts))


Number of 'Ask HN' posts: 1744
Number of 'Show HN' posts: 1162
Number of other posts: 17194


Next, let's determine if ask posts or show posts receive more comments on average.

In [9]:
total_ask_comments = 0 

for row in ask_posts:
    num_comments = int(row[4])
    total_ask_comments += num_comments
    
avg_ask_comments  = total_ask_comments/len(ask_posts)
print('Average comments on Ask HN posts: {:.2f}'.format(avg_ask_comments))


total_show_comments = 0
for row in show_posts:
    num_comments_s = int(row[4])
    total_show_comments += num_comments_s

avg_show_comments  = total_show_comments/len(show_posts)
print('Average comments on Show HN comments: {:.2f}'.format(avg_show_comments))


Average comments on Ask HN posts: 14.04
Average comments on Show HN comments: 10.32


We can see that ask posts recieve more comments on average.

Since ask posts are more likely to receive comments, we'll focus our remaining analysis just on these posts. \
Next, we'll determine if ask posts created at a certain time are more likely to attract comments. 

Now we'll to calculate the amount of ask posts created per hour, along with the total amount of comments.

In [10]:
import datetime as dt 

result_list = []

for row in ask_posts:
     result_list.append([row[6],int(row[4])]) #appending 'created at' and 'number of comments'

counts_by_hour = {}
comments_by_hour = {}

for row in result_list:
    comments_count = row[1]
    date_str = row[0]
    date_dt = dt.datetime.strptime(date_str,"%m/%d/%Y %H:%M")
    hour_dt = date_dt.hour
    
    if hour_dt not in counts_by_hour:
        counts_by_hour[hour_dt] = 1
        comments_by_hour[hour_dt] = comments_count
    else:
        counts_by_hour[hour_dt] += 1
        comments_by_hour[hour_dt] += comments_count

So we created two dictionaries:
* counts_by_hour: contains the number of ask posts created during each hour of the day.
* comments_by_hour: contains the corresponding number of comments ask posts created at each hour received.  

Next, we'll use these two dictionaries to calculate the average number of comments for posts created during each hour of the day.

In [11]:
avg_by_hour = []

for key in counts_by_hour:
    avg_posts = comments_by_hour[key]/counts_by_hour[key]
    avg_by_hour.append([key,avg_posts])

In [12]:
avg_by_hour

[[9, 5.5777777777777775],
 [13, 14.741176470588234],
 [10, 13.440677966101696],
 [14, 13.233644859813085],
 [16, 16.796296296296298],
 [23, 7.985294117647059],
 [12, 9.41095890410959],
 [17, 11.46],
 [15, 38.5948275862069],
 [21, 16.009174311926607],
 [20, 21.525],
 [2, 23.810344827586206],
 [18, 13.20183486238532],
 [3, 7.796296296296297],
 [5, 10.08695652173913],
 [19, 10.8],
 [1, 11.383333333333333],
 [22, 6.746478873239437],
 [8, 10.25],
 [4, 7.170212765957447],
 [0, 8.127272727272727],
 [6, 9.022727272727273],
 [7, 7.852941176470588],
 [11, 11.051724137931034]]

Although we now have the results we need, this format makes it hard to identify the hours with the highest values. Let's finish by sorting the list of lists and printing the five highest values in a format that's easier to read.

In [13]:
swap_avg_by_hour = []

for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])
    
print(swap_avg_by_hour)

[[5.5777777777777775, 9], [14.741176470588234, 13], [13.440677966101696, 10], [13.233644859813085, 14], [16.796296296296298, 16], [7.985294117647059, 23], [9.41095890410959, 12], [11.46, 17], [38.5948275862069, 15], [16.009174311926607, 21], [21.525, 20], [23.810344827586206, 2], [13.20183486238532, 18], [7.796296296296297, 3], [10.08695652173913, 5], [10.8, 19], [11.383333333333333, 1], [6.746478873239437, 22], [10.25, 8], [7.170212765957447, 4], [8.127272727272727, 0], [9.022727272727273, 6], [7.852941176470588, 7], [11.051724137931034, 11]]


In [14]:
#sorting swap_avg_by_hour in descending order
sorted_swap = sorted(swap_avg_by_hour, reverse = True)

print("Top 5 Hours for Ask Posts Comments")

for row in sorted_swap[:5]:
    hour_formatted = dt.datetime.strptime(str(row[1]),'%H')
    hour_formatted = hour_formatted.strftime('%H:%M')

    print('{}: {:.2f} average comments per post'.format(hour_formatted,row[0]))

Top 5 Hours for Ask Posts Comments
17:00: 38.59 average comments per post
04:00: 23.81 average comments per post
22:00: 21.52 average comments per post
18:00: 16.80 average comments per post
23:00: 16.01 average comments per post


And now we can see when we should create a post to have a higher chance of receiving comments. 

# Conclusion 

So in this project we reformatted an cleaned the data and analyzed it. And now we now that "Ask HN" posts have more cooments that "Show HN". And we also know that the best time to create your post is 17:00 (EST).